In [1]:
from vierlinden.config import model_output_path
from vierlinden.data.loader import VierlindenDataProcessor
from vierlinden.model.model import NHitsTrainingWrapper

batch_size = 32
num_workers = 18

def main(arg : str):
    dp = VierlindenDataProcessor()
    df = dp.load_processed_data()
    df = dp.prepare_for_target(df, arg)
    
    training_df, test_df = dp.split_data(df)
    
    nhits_wrapper = NHitsTrainingWrapper(training_df, arg, batch_size, num_workers)
    optimal_lr = nhits_wrapper.find_optimal_learningrate()
    
    print(f"Optimal learning rate for {arg}: {optimal_lr}")
    
    trainer = nhits_wrapper.train(optimal_lr, seed = 42)
    
    return trainer, test_df, nhits_wrapper

/storage/.venv/lib/python3.10/site-packages/pytorch_forecasting/models/base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
trainer, test_df, nhits_wrapper = main('Kaiserstr_outflow [l/s]')

INFO:vierlinden.data.loader:Data loaded successfully from /storage/projects/RIWWER/data/Vierlinden
INFO:vierlinden.data.loader:Sensor and target data merged successfully.
INFO:vierlinden.data.loader:NaN values processed successfully.
INFO:vierlinden.data.loader:Data loaded and processed successfully.


INFO:vierlinden.model.model:Training and validation data and data loaders created successfully.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/storage/.venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:197: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/storage/.venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:197: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Finding best initial lr: 100%|██████████| 100/100 [00:03<00:00, 30.26it/s]
Learning rate set to 0.00

Optimal learning rate for Kaiserstr_outflow [l/s]: 0.0019952623149688802


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type           | Params
---------------------------------------------------
0 | loss            | MASE           | 0     
1 | logging_metrics | ModuleList     | 0     
2 | embeddings      | MultiEmbedding | 0     
3 | model           | NHiTS          | 872 K 
---------------------------------------------------
872 K     Trainable params
0         Non-trainable params
872 K     Total params
3.488     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 193/193 [00:34<00:00,  5.62it/s, v_num=1, train_loss_step=2.140, val_loss=1.060, train_loss_epoch=1.510]

...

Swapping scheduler `ReduceLROnPlateau` for `SWALR`


Epoch 10: 100%|██████████| 193/193 [00:37<00:00,  5.21it/s, v_num=1, train_loss_step=0.662, val_loss=1.150, train_loss_epoch=1.250]

Monitored metric val_loss did not improve in the last 10 records. Best score: 1.061. Signaling Trainer to stop.


Epoch 10: 100%|██████████| 193/193 [00:37<00:00,  5.17it/s, v_num=1, train_loss_step=0.662, val_loss=1.150, train_loss_epoch=1.250]


INFO:vierlinden.model.model:Training procedure completed.


In [4]:
trainer.save_checkpoint(model_output_path + "/" + 'NHits_Vierlinden_Kaiserstr.ckpt')